<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 11 AutoGluon</h4>
</center>

In [2]:
# !pip install autogluon
# !pip install tslearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [7]:
df_ventas_product = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product.csv", header = None)
df_ventas_product.columns = ["timestamp", "item_id", "target"]
df_ventas_product.head(5)

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [18]:
df_product = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_products.csv", header = None)
df_product.columns = ["item_id", "cat1", "cat2", "cat3", "brand", "sku_size"]
df_product.head(5)

,item_id,cat1,cat2,cat3,brand,sku_size
0,20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
1,20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
2,20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
3,20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
4,20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [32]:
df_product = df_product[["item_id", "cat2"]]
df_product["cat2"] = df_product["cat2"].str.replace(" ", "_")
print(df_product["cat2"].unique())
len(df_product["cat2"].unique())

['ROPA_LAVADO' 'ROPA_MANCHAS' 'VAJILLA' 'PROFESIONAL' 'DEOS' 'CABELLO'
 'SOPAS_Y_CALDOS' 'HOGAR' 'DENTAL' 'ROPA_ACONDICIONADOR' 'PIEL2' 'PIEL1'
 'ADEREZOS' 'TE' 'OTROS']


15

In [30]:
df_product.groupby(["cat2"])["item_id"].nunique().reset_index().rename(columns = {"item_id":"q_items"}).sort_values(by = "q_items")

,cat2,q_items
11,ROPA_MANCHAS,2
13,TE,6
5,OTROS,9
2,DENTAL,14
8,PROFESIONAL,15
9,ROPA_ACONDICIONADOR,21
14,VAJILLA,33
6,PIEL1,47
0,ADEREZOS,49
10,ROPA_LAVADO,53


In [21]:
df = pd.merge(df_ventas_product, df_product[["item_id", "cat2"]], how = "left", on = "item_id")

In [24]:
categories = df['cat2'].drop_duplicates().tolist()

for category in categories:
    globals()[f'df_cat_{category}'] = df[df['cat2'] == category].copy()
    del globals()[f'df_cat_{category}']['cat2']  # Eliminar la columna cat2

In [34]:
df_cat_ROPA_LAVADO

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
8,2017-01-01,20009,378.08172
9,2017-01-01,20010,441.25546
10,2017-01-01,20011,489.25422
...,...,...,...
22025,2019-12-01,20611,9.19572
22050,2019-12-01,20649,14.79183
22076,2019-12-01,20682,5.35080
22081,2019-12-01,20691,7.99868


In [33]:
# Lista para almacenar los objetos TimeSeriesDataFrame
time_series_dataframes = []

# Crear un bucle for que itere sobre cada categoría única en cat2
for category in categories:
    # Construir el nombre de la variable del dataframe de la categoría actual
    df_name = f"df_cat_{category}"
    # Verificar si la variable existe en el entorno actual (recomendado usar locals())
    if df_name in locals():
        # Obtener el dataframe de la categoría actual
        df_categoria = locals()[df_name]
        # Crear un objeto TimeSeriesDataFrame con el dataframe actual
        data = TimeSeriesDataFrame(df_categoria)
        # Agregar el objeto TimeSeriesDataFrame a la lista
        time_series_dataframes.append(data)
        # Opcional: Asignar el objeto a una variable individual si es necesario
        globals()[f"data_{category}"] = data

In [35]:
data_ROPA_LAVADO

,,target
item_id,timestamp,
20001,2017-01-01,934.77222
20002,2017-01-01,550.15707
20009,2017-01-01,378.08172
20010,2017-01-01,441.25546
20011,2017-01-01,489.25422
...,...,...
20611,2019-12-01,9.19572
20649,2019-12-01,14.79183
20682,2019-12-01,5.35080


In [37]:
# Lista para almacenar los predictores entrenados
predictores = []

# Iterar sobre cada categoría y crear el predictor correspondiente
for category in categories:
    # Construir el nombre de la variable del objeto TimeSeriesDataFrame
    data_name = f"data_{category}"
    # Verificar si la variable existe en el entorno global (recomendado usar globals())
    if data_name in globals():
        # Obtener el objeto TimeSeriesDataFrame correspondiente
        data = globals()[data_name]
        # Crear el predictor de series temporales
        predictor_name = f"predictor_{category}"
        print(predictor_name)
        predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M").fit(data)
        # Agregar el predictor a la lista
        predictores.append(predictor)
        # Opcional: Asignar el predictor a una variable individual si es necesario
        globals()[predictor_name] = predictor

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240621_194249'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       0.55 GB / 7.88 GB (7.0%)
Disk Space Avail:   5.57 GB / 110.21 GB (5.1%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequency 'None' has been resam

predictor_ROPA_LAVADO


Provided train_data has 1533 rows (NaN fraction=0.1%), 53 time series. Median time series length is 36 (min=4, max=36). 
	Removing 4 short time series from train_data. Only series with length >= 7 will be used for training.
	After filtering, train_data has 1513 rows (NaN fraction=0.1%), 49 time series. Median time series length is 36 (min=7, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-06-21 16:42:51
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-0.2038       = Validation score (-WQL)
	0.01    

predictor_ADEREZOS


Provided train_data has 1526 rows, 49 time series. Median time series length is 36 (min=5, max=36). 
	Removing 4 short time series from train_data. Only series with length >= 7 will be used for training.
	After filtering, train_data has 1505 rows, 45 time series. Median time series length is 36 (min=14, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-06-21 16:53:25
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-0.2317       = Validation score (-WQL)
	0.03    s     = Training runtime
	6.93    s    

...

Provided train_data has 1181 rows (NaN fraction=0.2%), 47 time series. Median time series length is 36 (min=6, max=36). 
	Removing 3 short time series from train_data. Only series with length >= 7 will be used for training.
	After filtering, train_data has 1163 rows (NaN fraction=0.2%), 44 time series. Median time series length is 36 (min=7, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-06-21 18:59:36
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-0.3770       = Validation score (-WQL)
	0.02    

predictor_DENTAL


Provided train_data has 453 rows, 14 time series. Median time series length is 36 (min=15, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-06-21 19:10:17
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-0.4839       = Validation score (-WQL)
	0.00    s     = Training runtime
	3.28    s     = Validation (prediction) runtime
Training timeseries model RecursiveTabular. 
	-0.9167       = Validation score (-WQL)
	1.78    s     = Training runtime
	0.12    s     = Validation (prediction) runtime
Training t

predictor_TE


Provided train_data has 216 rows (NaN fraction=0.5%), 6 time series. Median time series length is 36 (min=36, max=36). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-06-21 19:17:49
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-1.9833       = Validation score (-WQL)
	0.01    s     = Training runtime
	3.14    s     = Validation (prediction) runtime
Training timeseries model RecursiveTabular. 
	-2.0882       = Validation score (-WQL)
	1.34    s     = Training runtime
	0.08    s     = Validation (prediction)

In [46]:
categories

['ROPA_LAVADO',
 'ADEREZOS',
 'VAJILLA',
 'ROPA_ACONDICIONADOR',
 'HOGAR',
 'SOPAS_Y_CALDOS',
 'ROPA_MANCHAS',
 'CABELLO',
 'PIEL2',
 'OTROS',
 'DEOS',
 'PROFESIONAL',
 'PIEL1',
 'DENTAL',
 'TE']

In [49]:
prediccion_ROPA_LAVADO = predictor_ROPA_LAVADO.predict(data_ROPA_LAVADO)
prediccion_ADEREZOS = predictor_ADEREZOS.predict(data_ADEREZOS)
prediccion_VAJILLA = predictor_VAJILLA.predict(data_VAJILLA)
prediccion_ROPA_ACONDICIONADOR = predictor_ROPA_ACONDICIONADOR.predict(data_ROPA_ACONDICIONADOR)
prediccion_HOGAR = predictor_HOGAR.predict(data_HOGAR)
prediccion_SOPAS_Y_CALDOS = predictor_SOPAS_Y_CALDOS.predict(data_SOPAS_Y_CALDOS)
prediccion_ROPA_MANCHAS = predictor_ROPA_MANCHAS.predict(data_ROPA_MANCHAS)
prediccion_CABELLO = predictor_CABELLO.predict(data_CABELLO)
prediccion_PIEL2 = predictor_PIEL2.predict(data_PIEL2)
prediccion_OTROS = predictor_OTROS.predict(data_OTROS)
prediccion_DEOS = predictor_DEOS.predict(data_DEOS)
prediccion_PROFESIONAL = predictor_PROFESIONAL.predict(data_PROFESIONAL)
prediccion_PIEL1 = predictor_PIEL1.predict(data_PIEL1)
prediccion_DENTAL = predictor_DENTAL.predict(data_DENTAL)
prediccion_TE = predictor_TE.predict(data_TE)

data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
data with frequency 'MS' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
data with frequency 'MS' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
data with frequency 'MS' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validatio

In [50]:
predictions = pd.concat([prediccion_ROPA_LAVADO,
                         prediccion_ADEREZOS,
                         prediccion_VAJILLA,
                         prediccion_ROPA_ACONDICIONADOR,
                         prediccion_HOGAR,
                         prediccion_SOPAS_Y_CALDOS,
                         prediccion_ROPA_MANCHAS,
                         prediccion_CABELLO,
                         prediccion_PIEL2,
                         prediccion_OTROS,
                         prediccion_DEOS,
                         prediccion_PROFESIONAL,
                         prediccion_PIEL1,
                         prediccion_DENTAL,
                         prediccion_TE])

In [51]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1315.717970
3,20002,1129.116252
5,20009,489.477884
7,20010,416.303423
9,20011,375.301385


In [53]:
len(predictions_v1.product_id.unique())

780

In [54]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_cluster_cat2.csv", sep = ",", index = False)